## Interactive Plotting with Bokeh
* Author: Johannes Maucher
* Last update: 12.06.2020

In [1]:
import pandas as pd
import numpy as np
#import bokeh
from IPython.display import display
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource,HoverTool

import json

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
from xml.etree.ElementTree import fromstring
from time import strptime, strftime
from optparse import OptionParser
import sys
import re

In [4]:
def findtext(e, name, default=None):
    """
    findtext
    helper function to find sub-element of e with given name and
    return text value
    returns default=None if sub-element isn't found
    """
    try:
        return e.find(name).text
    except:
        return default

In [5]:
def parsetcx(xml):
    """
    parsetcx
    parses tcx data, returning a list of all Trackpoints where each
    point is a tuple of 
      (activity, lap, timestamp, seconds, lat, long, alt, dist, heart, cad)
    xml is a string of tcx data
    """

    # remove xml namespace (xmlns="...") to simplify finds
    # note: do this using ET._namespace_map instead?
    # see http://effbot.org/zone/element-namespaces.htm
    xml = re.sub('xmlns=".*?"','',xml)

    # parse xml
    tcx=fromstring(xml)

    activity = tcx.find('.//Activity').attrib['Sport']
    sample=1
    lapnum=1
    points=[]
    for lap in tcx.findall('.//Lap/'):
        for point in lap.findall('.//Trackpoint'):
            # time, both as string and in seconds GMT
            # note: adjust for timezone?
            timestamp = findtext(point, 'Time')
            if timestamp:
                time = strftime('%X', strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f'))
                date = strftime('%x', strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f'))
                #print(strptime(timestamp[:-1],'%Y-%m-%dT%H:%M:%S.%f')[3])
            else:
                time = None
            # cummulative distance
            dist = findtext(point, 'DistanceMeters')
                
            # latitude and longitude
            position = point.find('Position')
            lat = findtext(position, 'LatitudeDegrees')
            long = findtext(position, 'LongitudeDegrees')

            # altitude
            alt = float(findtext(point, 'AltitudeMeters',0))
            
            # heart rate
            heart = int(findtext(point.find('HeartRateBpm'),'Value',0))

            # cadence
            cad = int(findtext(point, 'Cadence',0))

            # append to list of points
            points.append((sample,
                           activity,
                           lapnum,
                           date, 
                           time, 
                           lat,
                           long,
                           alt,
                           dist,
                           heart,
                           cad))
            sample+=1
        # next lap
        lapnum+=1
    return points

In [6]:
columnIndex={"sample":0,"date":3,"time":4,'lat':5,"long":6,"alt":7,"dist":8,"heart":9,"cad":10}

In [7]:
def createDataframe(pointslist,columnIndex):
    datadict={}
    for col in columnIndex.keys():
        datadict[col]=[]
    for row in pointslist:
        for col in columnIndex.keys(): 
            value=row[columnIndex[col]]
            if value == None:
                try:
                    value = datadict[col][-1]
                except:
                    value=0
            datadict[col].append(value)
    dataframe = pd.DataFrame(data=datadict,columns=columnIndex.keys(),index=datadict["sample"])
    #return dataframe.drop(['sample'],axis=1)
    return dataframe

In [8]:
istream = open('tcxfiles/trainingdata2018-04-07.tcx','r',encoding='utf-8')
# read xml contents and parse tcx
xml = istream.read()
points = parsetcx(xml)

In [9]:
trainingDF=createDataframe(points,columnIndex)

In [10]:
print(trainingDF.head())

   sample      date      time          lat        long  alt  \
1       1  04/07/18  14:44:20  48.70610833  9.11079167  0.0   
2       2  04/07/18  14:44:21  48.70609833  9.11081667  0.0   
3       3  04/07/18  14:44:22  48.70608667  9.11083833  0.0   
4       4  04/07/18  14:44:23  48.70607333  9.11085333  0.0   
5       5  04/07/18  14:44:24    48.706055  9.11085833  0.0   

                 dist  heart  cad  
1   0.800000011920929     80    0  
2  0.8999999761581421     86    0  
3   1.100000023841858     94    0  
4   3.299999952316284     98    0  
5   5.800000190734863    101    0  


In [11]:
HRZ=[93,112,130,149,167,186]
HRC=["grey","blue","green","yellow","red"]

In [12]:
from bokeh.layouts import gridplot
from bokeh.models import BoxAnnotation, Range1d

source = ColumnDataSource(trainingDF)

options = dict(plot_width=800, plot_height=300,
               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select,reset")

p1 = figure(title="heartrate over time", **options)
p1.line("sample","heart", color="blue", source=source)
for i,col in enumerate(HRC):
    p1.add_layout(BoxAnnotation(bottom=HRZ[i],top=HRZ[i+1], fill_alpha=0.3, fill_color=col))

p2 = figure(title="cadence over time",x_range=p1.x_range,y_range=Range1d(60,90), **options)
p2.line("sample","cad", color="green", source=source)

p3 = figure(title="altitude over time",x_range=p1.x_range,y_range=Range1d(300,600), **options)
p3.line("sample","alt", color="red", source=source)


p = gridplot([[p1],[p2],[p3]], toolbar_location="right")

show(p)

In [74]:
import folium

In [81]:
m = folium.Map(location=[float(trainingDF["lat"][1]), float(trainingDF["long"][1])],zoom_start=13)

In [82]:
locdata=[]
for i in range(trainingDF.shape[0]):
    locdata.append((float(trainingDF["lat"][i+1]),float(trainingDF["long"][i+1])))

In [86]:
track=folium.features.ColorLine(positions=locdata,weight=3,opacity=0.8,colors=range(len(path)))

In [87]:
track.add_to(m)

In [88]:
m